In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
from nilearn.plotting import plot_img, plot_stat_map, view_img, plot_prob_atlas
from nilearn.regions import connected_label_regions
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from nilearn.image import concat_imgs, mean_img, index_img, smooth_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
import matplotlib.pyplot as plt
from nilearn.plotting import plot_roi
from nilearn.maskers import NiftiMapsMasker, NiftiSpheresMasker
from nilearn.image import high_variance_confounds

/Users/luisalvarez/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Open a datasets directory. 
fd = os.open("/Users/luisalvarez/Documents/Datasets", os.O_RDONLY)

# Use os.fchdir() method to change the current dir/folder.
os.fchdir(fd)

# Safe check- Print current working directory
print("Current working dir : %s" % os.getcwd())

Current working dir : /Users/luisalvarez/Documents/Datasets


In [17]:
# Load data from sub-01. 

sub01_events = pd.read_csv('/Users/luisalvarez/Documents/GitHub/RM_Thesis_Neuroforecasting/ProcessedData/sub-01/sub-01_processed_events.csv', sep='\,')

# Separate into run 1 and run 2. 
sub01_run1 = sub01_events.iloc[:16, :]
sub01_run2 = sub01_events.iloc[16:, :]
sub01_run1.reset_index(drop=True, inplace=True)
sub01_run2.reset_index(drop=True, inplace=True)

# Should I adjust onsets for the time lag (the same way we did for the timecourse analysis)?

# Now, create different events df's for different modulation types.

sub01_run1_events_base = sub01_run1[["Type", "Onset"]]
sub01_run1_events_base.rename(columns={"Type": "trial_type", "Onset": "onset"}, inplace=True)
sub01_run1_events_base["duration"] = 5
# Adjust time lag... 
sub01_run2_events_base = sub01_run2[["Type", "Onset"]]
sub01_run2_events_base.rename(columns={"Type": "trial_type", "Onset": "onset"}, inplace=True)
sub01_run2_events_base["duration"] = 5

sub01_run1_events_PA = sub01_run1_events_base.copy()
sub01_run1_events_PA["modulation"] = sub01_run1["Pos_arousal_scaled"]

sub01_run2_events_PA = sub01_run2_events_base.copy()
sub01_run2_events_PA["modulation"] = sub01_run2["Pos_arousal_scaled"]

sub01_run1_events_PA.head()




/var/folders/0z/1yt2h6410kb7_mgghf4q28z00000gn/T/ipykernel_11452/2947252213.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sub01_events = pd.read_csv('/Users/luisalvarez/Documents/GitHub/RM_Thesis_Neuroforecasting/ProcessedData/sub-01/sub-01_processed_events.csv', sep='\,')
/var/folders/0z/1yt2h6410kb7_mgghf4q28z00000gn/T/ipykernel_11452/2947252213.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub01_run1_events_base.rename(columns={"Type": "trial_type", "Onset": "onset"}, inplace=True)
/var/folders/0z/1yt2h6410kb7_mgghf4q28z00000gn/T/ipykernel_11452/2947252213.py:17: SettingWithCopyWarning: 
A value

,trial_type,onset,duration,modulation
0,Horror,12.0,5,0.029333
1,Horror,60.0,5,0.292443
2,Comedy,108.0,5,-0.319109
3,Horror,152.0,5,-0.276443
4,Horror,196.0,5,0.029333


In [ ]:
# Load confound files. 
sub01_run1_confounds = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-01_desc-confounds_regressors.tsv"
sub01_run2_confounds = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-02_desc-confounds_regressors.tsv"

sub01_run1_confounds = pd.read_csv(sub01_run1_confounds, sep='\t')
sub01_run2_confounds = pd.read_csv(sub01_run2_confounds, sep='\t')

# Define default confounds and 
default_confounds = ["white_matter", "csf", "csf_wm", "framewise_displacement", "dvars", "rmsd", "tcompcor",     
                    'w_comp_cor_00', 'w_comp_cor_01', 'w_comp_cor_02', 'w_comp_cor_03', 'w_comp_cor_04']

In [19]:
# Load sub01 functional data.
sub01_run1_func = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
sub01_run2_func = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

# Load mask file.
sub01_run1_mask = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
sub01_run2_mask = "MovieData_BIDS_preproc/sub-" + "01" + "/func/sub-" + "01" + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"



In [ ]:
# Create first level model.
first_level_model = FirstLevelModel(t_r=2.0, 
                                    hrf_model='spm', 
                                    high_pass=1/360, 
                                    smoothing_fwhm=4.0)